In [7]:
#!pip3 install --user matplotlib

import sys


import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [8]:
aside=np.array([mpimg.imread(x) for x in glob.glob('/nfs/general/shared/KeyA/*.jpg')])
bside=np.array([mpimg.imread(x) for x in glob.glob('/nfs/general/shared/KeyD/*.jpg')])
y=np.zeros(aside.shape[0] + bside.shape[0])
y[:aside.shape[0]]=1.0
y[aside.shape[0]:]=0.0
X=np.concatenate((aside,bside),axis=0)
X=X.reshape(X.shape[0],-1)
kImgShape=aside.shape[1:]

def XtoImgShape(Xline):
    return Xline.reshape(aside.shape[1:])

def PlotSparseness(x):
    plt.imshow(np.where(x!=0,255,0), interpolation='nearest')
    
def GenSingleChannelDicts(s, windowSizes):
    for windowSize in windowSizes:
        for x in range(0, s[0], windowSize):
            for y in range(0, s[1], windowSize):
                r=np.zeros(s)
                r[x:x+windowSize,y:y+windowSize]=1
                yield r

def SpreadAcrossColors(mask):
    for i in [0,1,2]:
        r=np.zeros((*mask.shape,3))
        r[:,:,i]=mask
        yield r

def GenDict(s, windowSize):
    for d in GenSingleChannelDicts(s, windowSize):
        for m in SpreadAcrossColors(d):
            yield m

def GenDictSize(s, windowSize):
    size=0
    for d in GenSingleChannelDicts(s, windowSize):
        for m in SpreadAcrossColors(d):
            size += 1
    return size


In [ ]:
# sparse_mats=[sparse.coo_matrix(x.astype(np.float32).reshape((-1,1))) for x in GenDict(kImgShape[:2],[8,16,32])]
# proj=sparse.hstack(sparse_mats)
# Xp=X.dot(proj)
# #proj=np.array([x for x in GenDict(kImgShape[:2],[16,32])])
# #Xp=np.matmul(X, proj.reshape((proj.shape[0],-1)).transpose())
# #proj=sparse.hstack([sparse.csr_matrix(x.astype(np.float32).reshape((-1,1))) for x in GenDict(kImgShape[:2],[8,16,32])], format='csr')

# #Xp=np.matmul(X, proj)
# Xp=X

# # foo=PCA(n_components=2).fit_transform(Xp.transpose())
# # plt.scatter(foo[:,0],foo[:,1],color='r')
# # plt.scatter(foo[:10,0],foo[:10,1],color='g')


# #clf = linear_model.Lasso(alpha=0.1)
# #clf.fit(X,y)

# eps = 1e-3
# #alphas_lasso, coefs_lasso, coefs = linear_model.lasso_path(X,Y, eps=eps, verbose=True,  alphas=np.logspace(-4, 1, 10))

# alphas_lasso, coefs_lasso, coefs = linear_model.lasso_path(Xp,y,eps=eps, verbose=True,  alphas=np.logspace(-4, 4, 10))

# #alphas_lars, coefs_lars, coefs = linear_model.lars_path(X, y, method="lasso", verbose=True)
# #alphas_lars

In [34]:
#reg.predict(X)
reg.predict(X)

array([ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.14450854e+00,  1.00000000e+00,  4.44089210e-15,  5.40177785e-01,
       -8.65973959e-15,  1.11022302e-15,  1.26565425e-14,  1.53210777e-14,
       -1.26565425e-14,  7.99360578e-15, -3.10549831e-01, -3.10862447e-15,
        8.88178420e-16,  2.73114864e-14, -1.99840144e-15, -1.19904087e-14,
        2.66453526e-15, -2.44249065e-15,  1.81814534e-01,  1.17498679e+00,
       -7.32747196e-15,  1.33226763e-15,  2.22044605e-15,  1.06581410e-14,
        3.33066907e-15,  2.88657986e-15, -1.73194792e-14, -5.77315973e-15,
        9.99200722e-15,  9.54791801e-15, -1.75415238e-14, -8.21565038e-15,
       -8.43769499e-15, -

In [17]:
from sklearn.linear_model import OrthogonalMatchingPursuit

reg = OrthogonalMatchingPursuit(normalize=False, n_nonzero_coefs=1000).fit(X, y)

/home/geli/.local/lib/python3.8/site-packages/sklearn/linear_model/_omp.py:419: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(


In [25]:
# PlotSparseness(XtoImgShape(reg.coef_))

reshapedCoefs=reg.coef_.reshape(kImgShape)
x_,y_,c_=np.nonzero(reshapedCoefs)


max(x_)
max(c_)


2

In [27]:

def SaveModel(reg, fname):
    import array
    reshapedCoefs=reg.coef_.reshape(kImgShape)
    x_,y_,c_=np.nonzero(reshapedCoefs)
    coefs_=reshapedCoefs[x_,y_,c_]
    f=open(fname,'wb')

    sizeArray=array.array('l')
    sizeArray.fromlist(list(kImgShape)+[len(x_)])
    f.write(sizeArray.tobytes())

    indexArray=array.array('l')
    tmp = list(x_)+list(y_)+list(c_)
    indexArray.fromlist(tmp)
    print(len(tmp))
    f.write(indexArray.tobytes())

    coefArray=array.array('f')
    coefArray.append(reg.intercept_)
    coefArray.extend(list(coefs_))
    f.write(coefArray.tobytes())

    f.close()

SaveModel(reg, '/nfs/general/shared/model')
#     f.close()


#X.dot(reg.coef_)
# lambda_max=np.max(np.abs(X.transpose().dot(y)))/y.size
# print(lambda_max)
# l=linear_model.Lasso(alpha=lambda_max*0.01)
# r=l.fit(X,y)


#np.sum(coefs_lasso[:,-1] != 0.0)
#PlotSparseness(XtoImgShape(coefs_lasso[:,-1]))

#coefs_lasso.shape

#plt.imshow(np.max(proj[coefs_lasso[:,2]!=0,:,:,:],axis=0), interpolation='nearest')
#print('num coefs={}'.format(np.sum(np.abs(l.coef_) > 0.0)))

# PlotSparseness(XtoImgShape(X[0,:]))

#len(coefs_)



303


In [ ]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

#plt.imshow(np.max(proj[coefs_lasso[:,1]!=0,:,:,:],axis=0), interpolation='nearest')

#coefs_lasso[coefs_lasso[:,1]!=0,1]

In [1]:
kImgShape

NameError: name 'kImgShape' is not defined

In [ ]:
#np.sum(coefs_lasso!=0,axis=0)

In [ ]:
#help(linear_model.lasso_path)

In [ ]:
#from sklearn.decomposition import sparse_encode 

In [ ]:
#help(sparse_encode)

In [ ]:
# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA

# reduced_data = PCA(n_components=2).fit_transform(X)

In [ ]:
# plt.scatter(reduced_data[:,0],reduced_data[:,1],color='r')
# plt.scatter(reduced_data[:10,0],reduced_data[:10,1],color='b')

In [ ]:
BASE_DIR='/nfs/general/shared'
A_FILES=glob.glob('{}/KeyA/*.jpg'.format(BASE_DIR))
D_FILES=glob.glob('{}/KeyD/*.jpg'.format(BASE_DIR))
[x.replace(BASE_DIR, '') for x in  D_FILES]
